In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("..")

In [117]:
import numpy as np
import torch
import transformers

In [39]:
from transformers import BertLMHeadModel, BertTokenizerFast

In [16]:
import settings
from funcs.data_module import DataModule
from funcs.utils import find_project_root

In [7]:
ROOT = find_project_root()

In [8]:
MODEL_NAME = settings.chinese_bert_model_name

In [11]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
tokenizer

In [40]:
model = BertLMHeadModel.from_pretrained(MODEL_NAME, return_dict=True)
model

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [17]:
data_module = DataModule()
data_module.setup()

2020-10-20 21:09:48.100 | INFO     | funcs.data_module:__init__:46 - data module hparams: {'max_tokenization_length': 64, 'batch_size': 64, 'num_workers': 2, 'min_word_frequency': 8}
2020-10-20 21:09:48.102 | INFO     | funcs.data_module:setup:49 - Loading train dataset
2020-10-20 21:09:48.104 | INFO     | funcs.data_module:get_dataset:84 - load from path: /work/ik18445/projects/yapg/datasets/output/poetry_64.pt


In [18]:
data_loader = data_module.train_dataloader()
print(len(data_loader))
for idx, batch in enumerate(data_loader):
    if idx == 2:
        data_batch = batch
        break
data_batch

385


[tensor([[ 101, 2526,  704,  ...,    0,    0,    0],
         [ 101, 5874, 2526,  ...,    0,    0,    0],
         [ 101,  736, 4371,  ...,    0,    0,    0],
         ...,
         [ 101, 1066, 6457,  ...,    0,    0,    0],
         [ 101, 1760, 4128,  ...,    0,    0,    0],
         [ 101, 5428, 7731,  ...,    0,    0,    0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]])]

In [22]:
inputs = {
    "input_ids": data_batch[0],
    "attention_mask": data_batch[1],
    "token_type_ids": data_batch[2],
    "labels": data_batch[0],
}
outputs = model(**inputs)
outputs

MaskedLMOutput(loss=tensor(7.4247, grad_fn=<NllLossBackward>), logits=tensor([[[ -7.4897,  -7.3078,  -7.3892,  ...,  -6.1555,  -6.8452,  -6.3288],
         [ -7.0119,  -6.8831,  -6.8382,  ...,  -5.5850,  -5.7504,  -5.4461],
         [ -8.4322,  -8.9203,  -9.2589,  ...,  -6.0365,  -7.4740,  -3.4020],
         ...,
         [ -6.7556,  -6.7236,  -7.0349,  ...,  -5.6988,  -7.0706,  -5.7675],
         [ -6.5486,  -6.3824,  -6.7158,  ...,  -5.6458,  -7.0196,  -5.6393],
         [ -7.0738,  -6.9013,  -7.3119,  ...,  -5.9904,  -6.8210,  -5.0939]],

        [[ -7.1489,  -7.0483,  -7.0265,  ...,  -6.4867,  -6.8877,  -5.7832],
         [ -7.2843,  -7.1744,  -7.1128,  ...,  -6.1080,  -6.5044,  -5.4240],
         [ -6.2073,  -6.2613,  -6.3986,  ...,  -6.2980,  -7.1251,  -5.5918],
         ...,
         [ -5.5722,  -5.4419,  -5.5996,  ...,  -5.3593,  -6.5668,  -3.6356],
         [ -5.7025,  -5.5547,  -5.6267,  ...,  -5.2922,  -6.1632,  -4.5116],
         [ -5.6294,  -5.3395,  -5.4290,  ...,  -5.118

In [23]:
outputs.keys()

odict_keys(['loss', 'logits'])

In [60]:
TEXT = "生活是"
sentence = f"{TEXT}{tokenizer.mask_token}"
encodings = tokenizer(sentence, return_tensors="pt")
encodings

{'input_ids': tensor([[ 101, 4495, 3833, 3221,  103,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [61]:
inputs = {
    "input_ids": encodings["input_ids"],
    "attention_mask": encodings["attention_mask"],
    "token_type_ids": encodings["token_type_ids"],
    "labels": encodings["input_ids"],
}
outputs = model(**inputs)
outputs.keys()

odict_keys(['loss', 'logits'])

In [62]:
mask_token_index = torch.where(encodings["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_index

tensor([4])

In [63]:
tokenizer.mask_token_id

103

In [64]:
logits = outputs["logits"]
logits

tensor([[[-7.9595, -7.8608, -7.9207,  ..., -6.9487, -7.1159, -7.0181],
         [-8.1631, -8.0284, -8.0136,  ..., -6.9275, -7.1323, -6.6380],
         [-8.7900, -8.6704, -8.5597,  ..., -7.0898, -8.7183, -7.7622],
         [-8.2436, -8.1421, -8.2977,  ..., -6.1147, -6.1333, -5.7758],
         [-7.8990, -7.5713, -7.6846,  ..., -4.6256, -5.5561, -4.9925],
         [-9.0508, -9.1447, -9.2875,  ..., -7.6092, -8.1554, -8.0123]]],
       grad_fn=<AddBackward0>)

In [76]:
mask_token_logits = logits[0, mask_token_index, :].detach()
mask_token_logits

tensor([[-7.8990, -7.5713, -7.6846,  ..., -4.6256, -5.5561, -4.9925]])

In [93]:
K = 30
top_k = torch.topk(mask_token_logits, K, dim=1)
top_k_logits = top_k[0][0]
top_k_probs = torch.nn.functional.softmax(top_k_logits)
top_k_indices = top_k[1][0]

/home/ik18445/miniconda3/envs/yapg/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


In [94]:
for idx, token in enumerate(top_k_indices.tolist()):
    filled_sentence = sentence.replace(tokenizer.mask_token, tokenizer.decode([token]))
    prob = top_k_probs[idx]
    print(f"sentence: {filled_sentence}\tprob: {prob}")

sentence: 生活是。	prob: 0.6666449308395386
sentence: 生活是！	prob: 0.18476134538650513
sentence: 生活是？	prob: 0.14617371559143066
sentence: 生活是，	prob: 0.0006420866120606661
sentence: 生活是...	prob: 0.00035333880805410445
sentence: 生活是：	prob: 0.00031461031176149845
sentence: 生活是人	prob: 0.0001470243587391451
sentence: 生活是吗	prob: 0.00014102619024924934
sentence: 生活是、	prob: 0.00013337582640815526
sentence: 生活是我	prob: 8.762655488681048e-05
sentence: 生活是嗎	prob: 6.456378469010815e-05
sentence: 生活是:	prob: 6.07806105108466e-05
sentence: 生活是.	prob: 5.6154309277189896e-05
sentence: 生活是,	prob: 4.761806849273853e-05
sentence: 生活是你	prob: 3.793793803197332e-05
sentence: 生活是"	prob: 3.672907405416481e-05
sentence: 生活是的	prob: 2.9054008336970583e-05
sentence: 生活是-	prob: 2.8676879082922824e-05
sentence: 生活是在	prob: 2.6001134756370448e-05
sentence: 生活是家	prob: 2.5144174287561327e-05
sentence: 生活是愛	prob: 2.396278068772517e-05
sentence: 生活是爱	prob: 2.3668275389354676e-05
sentence: 生活是么	prob: 2.239015157101676e-05
sentenc

In [96]:
token

1962

In [99]:
tokenizer(tokenizer.convert_ids_to_tokens(token))

{'input_ids': [101, 1962, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

TODO: a "poem" generator function

In [218]:
def get_logits(encodings):
    inputs = {
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"],
        "token_type_ids": encodings["token_type_ids"],
        "labels": encodings["input_ids"],
    }
    outputs = model(**inputs)
    logits = outputs["logits"].detach()
    return logits


def generate_text(
    tokenizer: BertTokenizerFast,
    model: BertLMHeadModel,
    starting_text: str,
    max_doc_length: int = 64,
    max_tokenization_length: int = 128,
    top_k: int = 100,
):

    mask_token_id = tokenizer.mask_token_id
    # starting_encodings = tokenizer(
    #     starting_text,
    #     truncation=True,
    #     padding="max_length",
    #     max_length=max_tokenization_length,
    #     return_tensors="pt",
    # )
    target_text = starting_text
    # target_ids = starting_encodings["input_ids"]
    i = 0
    while i < max_doc_length:
        print("run:", i)
        # print(target_ids)
        # print(tokenizer.decode(target_ids))
        encode_text = target_text + tokenizer.mask_token
        # encodings = tokenizer(target_text, return_tensors="pt")
        # encode_ids = target_ids + [mask_token_id]
        encodings = tokenizer(
            # tokenizer.convert_ids_to_tokens(encode_ids),
            encode_text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=max_tokenization_length,
        )
        logits = get_logits(encodings)
        # print(logits)
        mask_token_index = torch.where(
            encodings["input_ids"] == tokenizer.mask_token_id
        )[1]
        mask_token_logits = logits[0, mask_token_index, :]
        # print(mask_token_logits)
        top_k_candidates = torch.topk(mask_token_logits, top_k, dim=1)
        # try:
        #     top_k_logits = top_k_candidates[0][0]
        # except:
        #     print(mask_token_logits)
        #     print(top_k_candidates)
        #     break
        top_k_logits = top_k_candidates[0][0]
        top_k_probs = torch.nn.functional.softmax(top_k_logits).numpy()
        top_k_ids = top_k_candidates[1][0].tolist()
        # randomly select an elem from top_k_ids, based on their softmax prob
        target_id = np.random.choice(top_k_ids, p=top_k_probs)
        print(target_id)
        target_text = target_text + tokenizer.decode([target_id])
        print(target_text)
        i = i + 1
    # target_text = tokenizer.decode(target_ids)
    return target_text

In [220]:
starting_text = "冬天"
text = generate_text(tokenizer=tokenizer, model=model, starting_text=starting_text,)

run: 0


/home/ik18445/miniconda3/envs/yapg/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


511
冬天。
run: 1
511
冬天。。
run: 2
511
冬天。。。
run: 3
511
冬天。。。。
run: 4
511
冬天。。。。。
run: 5
511
冬天。。。。。。
run: 6
511
冬天。。。。。。。
run: 7
511
冬天。。。。。。。。
run: 8
511
冬天。。。。。。。。。
run: 9
511
冬天。。。。。。。。。。
run: 10
511
冬天。。。。。。。。。。。
run: 11
511
冬天。。。。。。。。。。。。
run: 12
511
冬天。。。。。。。。。。。。。
run: 13
511
冬天。。。。。。。。。。。。。。
run: 14
511
冬天。。。。。。。。。。。。。。。
run: 15
511
冬天。。。。。。。。。。。。。。。。
run: 16
511
冬天。。。。。。。。。。。。。。。。。
run: 17
511
冬天。。。。。。。。。。。。。。。。。。
run: 18
511
冬天。。。。。。。。。。。。。。。。。。。
run: 19
511
冬天。。。。。。。。。。。。。。。。。。。。
run: 20
519
冬天。。。。。。。。。。。。。。。。。。。。「
run: 21
6956
冬天。。。。。。。。。。。。。。。。。。。。「部
run: 22
7386
冬天。。。。。。。。。。。。。。。。。。。。「部隊
run: 23
520
冬天。。。。。。。。。。。。。。。。。。。。「部隊」
run: 24
511
冬天。。。。。。。。。。。。。。。。。。。。「部隊」。
run: 25
511
冬天。。。。。。。。。。。。。。。。。。。。「部隊」。。
run: 26
511
冬天。。。。。。。。。。。。。。。。。。。。「部隊」。。。
run: 27
511
冬天。。。。。。。。。。。。。。。。。。。。「部隊」。。。。
run: 28
511
冬天。。。。。。。。。。。。。。。。。。。。「部隊」。。。。。
run: 29
511
冬天。。。。。。。。。。。。。。。。。。。。「部隊」。。。。。。
run: 30
511
冬天。。。。。。。。。。。。。。。。。。。。「部隊」。。。。。。。
run: 31
511
冬天。。。。。。。。。。。。。。。。。。。。「部隊」。。。。。。。。
run: 32